# React Prompting without similar Problems

In [ ]:
import json
from functions import inference
from templates_english import *

In [ ]:
from langchain_core.messages import (
    BaseMessage,
    HumanMessage,
    ToolMessage,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langgraph.graph import END, StateGraph, START
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from typing import Annotated
from langchain_experimental.tools import PythonREPLTool

In [ ]:
python_repl_tool = PythonREPLTool()

In [ ]:
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    """Creating an agent"""
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

In [ ]:
def agent_node(state, agent, name, log):
    result = agent.invoke(state)
    log[name] = result["output"]
    return {"messages": [HumanMessage(content=result['output'], name=name)]}

In [ ]:
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

members = ["Noter", "Validator", "Explainer"]
system_prompt = (
    "You are a supervisor responsible for managing the dialogue between the following members: {members}."
    "Based on the user requests below, designate the next member who needs to act. Each member will perform a task,"
    "your job is to coordinate the work between Noter, Explainer, and Validator to produce a perfect problem analysis."
    "Validator checks if Noter's board work is correct, if not, they should return the correct answer to Noter who will then regenerate the board work according to the correct answer."
    "Once Noter regenerates the board work, pass it back to Validator for verification. Only when Validator confirms the accuracy of Noter's board work, move on to Explainer."
    "Report their results and status at each step. After the task is completed, please reply with “FINISH”."
    "In most cases, we start with Noter and continue to Validator to check if the answer given by noter is correct then decide who goes next."
)

options = ["FINISH"] + members

function_def = {
    "name": "route",
    "description": "choose which agent should go next",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "According to current conversation, which member should go next"
            "Or have we already got a perfect solution so we can FINISH?"
            "Choose from one of the following options：{options}",
        )
    ]
).partial(options=str(options), members=", ".join(members))

llm = ChatOpenAI(model='gpt-4o', max_tokens=1000)

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [ ]:
import functools
import operator
from typing import Sequence, TypedDict

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str
log = {}

noter_agent = create_agent(llm, [python_repl_tool], noter_template)
noter_node = functools.partial(agent_node, agent=noter_agent, name="Noter", log=log)

explainer_agent = create_agent(llm, [python_repl_tool], explainer_react_template)
explainer_node = functools.partial(agent_node, agent=explainer_agent, name="Explainer", log=log)

validator_agent = create_agent(llm, [python_repl_tool], validator_correctness_template)
validator_node = functools.partial(agent_node, agent=validator_agent, name="Validator", log=log)

def supervisor_node(state, log):
    result = supervisor_chain.invoke(state)
    log["Supervisor"] = result
    return result

workflow = StateGraph(AgentState)
workflow.add_node("Noter", noter_node)
workflow.add_node("Explainer", explainer_node)
workflow.add_node("Validator", validator_node)
workflow.add_node("Supervisor", functools.partial(supervisor_node, log=log))

In [ ]:
for member in members:
    workflow.add_edge(member, "Supervisor")

conditional_map = {k:k for k in members}
conditional_map['FINISH'] = END
workflow.add_conditional_edges("Supervisor", lambda x: x['next'], conditional_map)
workflow.add_edge(START, "Supervisor")

graph = workflow.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    pass

In [ ]:
import json
with open("test_problem_set.json", 'r') as file:
    problem_set = json.load(file)
problems = []
answers = []
for i in range(len(problem_set)):
    problems.append(problem_set[i]['problem'])
    answers.append(problem_set[i]['answer'])

In [ ]:
from langchain_core.output_parsers import StrOutputParser 
llm = ChatOpenAI(model='gpt-4o', max_tokens=1000)
parser = StrOutputParser()
prompt_translate = ChatPromptTemplate.from_template(chinese2eng_template)
prompt_reverse = ChatPromptTemplate.from_template(eng2chinese_template)
chain_translate = prompt_translate | llm | parser
chain_reverse = prompt_reverse | llm | parser

In [ ]:
# english_problems = []
# # english_answers = []
# for i in range(len(answers)):
#     output_problem = chain_translate.invoke({"problem": problems[i]})
#     # output_answer = chain_translate.invoke({"problem": answers[i]})
#     print(output_problem)
#     english_problems.append(output_problem)
#     # english_answers.append(output_answer)

In [ ]:
# write2 = []
# for i in range(len(english_problems)):
#     cur = {}
#     cur['problem'] = english_problems[i]
#     cur['answer'] = english_answers[i]
#     write2.append(cur)

# with open("test_problem_set_english.json", 'w') as file:
#     json.dump(write2, file, ensure_ascii=False, indent=4)

In [47]:
with open("test_problem_set_english.json", 'r') as file:
    tmp = json.load(file)
english_problems = []
english_answers = []
for ps in tmp:
    english_answers.append(ps['answer'])
    english_problems.append(ps['problem'])

In [53]:
log = []
for i in range(len(english_problems)):
    record = []
    problem = "### Problem to Solve: " + english_problems[i]
    answer = "### Reference Solution: " + english_answers[i]
    try:
        for s in graph.stream(
            {
                "messages": [
                    HumanMessage(content=problem),
                    HumanMessage(content=answer)
                ],
            },
            {"recursion_limit": 10},
        ):
            if "__end__" not in s:
                record.append(s)
                print(s)
                print("----")
        log.append(record)
    except Exception as e:
        log.append(["error", "error"])


{'Supervisor': {'next': 'Noter'}}
----
{'Noter': {'messages': [HumanMessage(content='Knowledge Point Explanation: This problem involves understanding different types of numbers including integers, positive real numbers, and irrational numbers. The key to solving it is to correctly classify each given number into the appropriate set based on its properties.\n\n### Board Writing One:\n(1) Set of integers: \n\\{ 0, -\\sqrt{9}, -10 \\}\n\n### Board Writing Two:\n(2) Set of positive real numbers: \n\\{ \\pi, 2.022 \\}\n\n### Board Writing Three:\n(3) Set of irrational numbers: \n\\{ \\pi, -1.1010010001\\ldots \\}\n\nFinal answer: The numbers have been correctly classified into their respective sets.', name='Noter')]}}
----
{'Supervisor': {'next': 'Validator'}}
----
{'Validator': {'messages': [HumanMessage(content="### Verification Process:\n\nLet's verify each of the classifications given by Noter.\n\n#### (1) Set of integers: { 0, -√9, -10 }\n\n- Integers are whole numbers that can be posi

In [52]:
log

[['error', 'error'],
 [{'Supervisor': {'next': 'Noter'}},
  {'Noter': {'messages': [HumanMessage(content='Knowledge Point Explanation: This problem involves solving inequalities. The key to solving it is to manipulate and rearrange the inequality to isolate \\( x \\), and then use the given solution set to determine the constraints on \\( a \\).\n\n### Board Writing One:\nRemoving the parentheses:\n\\[ ax - a > x + 1 - 2a \\]\n\nRearranging terms:\n\\[ ax - x > 1 - 2a + a \\]\n\nCombining like terms:\n\\[ (a - 1)x > 1 - a \\]\n\nGiven solution set:\n\\[ x < -1 \\]\n\nSubstituting \\( x < -1 \\):\n\\[ (a - 1)(-1) > 1 - a \\]\n\\[ -a + 1 > 1 - a \\]\n\nSimplifying:\n\\[ -a + 1 > 1 - a \\]\n\nThus:\n\\[ a - 1 < 0 \\]\n\\[ a < 1 \\]\n\n### Board Writing Two:\nSo the range of values for \\( a \\) is:\n\\[ a < 1 \\]', name='Noter')]}},
  {'Supervisor': {'next': 'Validator'}},
  {'Validator': {'messages': [HumanMessage(content="### Verification Process:\n\nLet's verify each step in the soluti

In [ ]:
with open('model_2_output.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

In [54]:
final_notes = []
final_expls = []
for i in range(len(log)):
    noter_log = []
    explainer_log = []
    # print(log[i])
    record = log[i]
    if record[0] == "error" and record[1] == "error":
        final_notes.append("Error")
        final_expls.append("Error")
        continue
    for item in record:
        for role in item:
            if role == "Supervisor" or role == "Validator": continue
            if role == "Noter":
                noter_log.append(item[role]['messages'][0].content)
            if role == "Explainer":
                explainer_log.append(item[role]['messages'][0].content)
    if len(noter_log): final_notes.append(noter_log[-1])
    else: final_notes.append("Error" + str(i))
    if len(explainer_log): final_expls.append(explainer_log[-1])
    else: final_expls.append("Error" + str(i))

In [55]:
final_notes

['Knowledge Point Explanation: This problem involves understanding different types of numbers including integers, positive real numbers, and irrational numbers. The key to solving it is to correctly classify each given number into the appropriate set based on its properties.\n\n### Board Writing One:\n(1) Set of integers: \n\\{ 0, -\\sqrt{9}, -10 \\}\n\n### Board Writing Two:\n(2) Set of positive real numbers: \n\\{ \\pi, 2.022 \\}\n\n### Board Writing Three:\n(3) Set of irrational numbers: \n\\{ \\pi, -1.1010010001\\ldots \\}\n\nFinal answer: The numbers have been correctly classified into their respective sets.',
 'Knowledge Point Explanation: This problem involves solving inequalities and understanding how the solution set of an inequality influences the range of the parameter \\(a\\). The key to solving it is manipulating and rearranging the inequality to isolate \\(a\\).\n\n### Board Writing One:\n\nGiven: \\( a(x-1) > x + 1 - 2a \\)\n\nRemove parentheses:\n\\[ ax - a > x + 1 - 2a

In [57]:
model_notes_chinese = []
for eng_p in final_notes:
    response = chain_reverse.invoke({"problem": eng_p})
    print(response)
    model_notes_chinese.append(response)

知识点解释：这个问题涉及理解不同类型的数字，包括整数、正实数和无理数。解决这个问题的关键是根据每个数字的属性正确地将其分类到适当的集合中。

### 板书一:
(1) 整数集合: 
\{ 0, -\sqrt{9}, -10 \}

### 板书二:
(2) 正实数集合: 
\{ \pi, 2.022 \}

### 板书三:
(3) 无理数集合: 
\{ \pi, -1.1010010001\ldots \}

最终答案: 这些数字已被正确分类到各自的集合中。
知识点解析：这个问题涉及解不等式并理解不等式的解集如何影响参数 \(a\) 的范围。解决这个问题的关键是操作和重新排列不等式以隔离 \(a\)。

### 板书一：

已知：\( a(x-1) > x + 1 - 2a \)

去掉括号：
\[ ax - a > x + 1 - 2a \]

重新排列项：
\[ ax - x > 1 - 2a + a \]

合并同类项：
\[ (a - 1)x > 1 - a \]

从解集 \( x < -1 \) 可以得出：
\[ a - 1 < 0 \]

因此：
\[ a < 1 \]

总结：\(a\) 的取值范围是 \( a < 1 \)。
```markdown
### Knowledge Point Explanation: 
这个问题涉及实数的一些运算，特别是使用一个定义的运算 \(a ⊗ b\)。解决这个问题的关键是应用给定的定义并适当地简化。

### Board Writing One: 
#### (1)
\[
3 ⊗ 4 = \frac{1}{3} - \frac{3 - 4}{3}
\]
\[
= \frac{1}{3} - \frac{-1}{3}
\]
\[
= \frac{1}{3} + \frac{1}{3}
\]
\[
= \frac{2}{3}
\]
所以，\(3 ⊗ 4\) 的值是 \(\frac{2}{3}\)。

### Board Writing Two:
#### (2)
根据题意，我们有
\[
2 ⊗ x = \frac{1}{2} - \frac{2 - x}{2} < 1
\]
清除分母：
\[
1 - (2 - x) < 2
\]
去掉括号：
\[
1 - 2 + x < 2
\]
整理并合并同类项：
\[
x <

In [58]:
model_expls_chinese = []
for eng_p in final_expls:
    response = chain_reverse.invoke({"problem": eng_p})
    print(response)
    model_expls_chinese.append(response)

让我们系统地分解如何将给定的数字分类到各自的集合：整数、正实数和无理数。

### 板书一：分类整数
#### 分析：
1. 整数是可以为正数、负数或零的整数。
2. 它们不包括分数或小数。

#### 步骤：
1. **0**：这是一个整数，因此是整数。
2. **-√9**：9的平方根是3。因此，-√9简化为-3，这是一个整数。
3. **-10**：这是一个整数，因此是整数。

#### 下一步：
识别不属于此集合的数字，并考虑将它们分类到其余的集合中。

### 板书二：分类正实数
#### 分析：
1. 正实数是大于零的数字。
2. 它们包括有理数和无理数，只要它们是正数。

#### 步骤：
1. **π**：π（pi）约为3.14159，这是正数。
2. **2.022**：这是一个正数。
3. **-10/3**：这是一个负数，因此不是正实数。
4. **-0.15**：这是一个负数，因此不是正实数。

#### 下一步：
识别不属于此集合的剩余数字以进一步分类。

### 板书三：分类无理数
#### 分析：
1. 无理数不能表示为两个整数的分数。
2. 它们的小数扩展是无限的且不重复的。

#### 步骤：
1. **π**：我们已经知道π是无理数。
2. **-1.101 001 000 1⋯**：这个数字是无限的且不重复，因此是无理数。

#### 下一步：
检查任何剩余的数字以确保它们被正确分类。

### 结论：
所有给定的数字现在已正确分类到它们所属的集合中：

1. **整数集合**：{ 0, -√9, -10 }
2. **正实数集合**：{ π, 2.022 }
3. **无理数集合**：{ π, -1.101 001 000 1⋯ }

这种方法确保了对分类过程和每种类型数字的属性有深入的理解。
让我们一步步分解问题和解决方案以更好地理解它。

### 解析一：分析问题：给定不等式 \(a(x-1) > x + 1 - 2a\)，确定 \(a\) 的取值范围。

a. 我们的起点是不等式 \(a(x-1) > x + 1 - 2a\)。

b. 我们需要简化这个不等式以便更容易分析。这涉及消除括号和重新排列项。

c. 为了简化不等式，我们首先去除括号。这告诉我们什么？

### 解析二：去除括号：从 \(a(x-1) > x 

In [59]:
pbl_set = []
for i in range(len(final_notes)):
    cur = {}
    cur['problem'] = problems[i]
    cur['Note'] = model_notes_chinese[i]
    cur['Expl'] = model_expls_chinese[i]
    pbl_set.append(cur)

with open("outputs/model_4_ouptut.json", 'w', encoding='utf-8') as file:
    json.dump(pbl_set, file, ensure_ascii=False, indent=4)

In [60]:
from langchain_core.output_parsers import StrOutputParser
async def check(note1: str, note2: str) -> bool:
    llm = ChatOpenAI(model='gpt-4o')
    sys_prompt =    r"""
                        你是一个题解检查者，你负责检查两个题解的最终答案是否一致。注意：题解不需要一模一样，
                        只要保证最后每一问的答案都完全一致即可。
                        如果答案一致：请输出是。如果答案不一致：请输出否。请确保输出一个字，不要输出“是“与”否”意外的内容

                        题解1:
                        {note1}
                        
                        题解2:
                        {note2}
                    """
    prompt = ChatPromptTemplate.from_template(sys_prompt)
    parser = StrOutputParser()
    chain = prompt | llm | parser

    response = chain.invoke(input={"note1": note1, "note2": note2})
    return response

In [61]:
scores = [0 for _ in range(len(model_expls_chinese))]
for i in range(len(model_expls_chinese)):
    output = await check(model_notes_chinese[i], answers[i])
    print(i)
    if output == "是":
        scores[i] = 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [62]:
sum(scores)

87

In [ ]:
print(final_expls[0])

In [ ]:
for problem in problems[:1]:
    cur_note = await inference({"problem": problem}, noter_template)
    react_expl = await inference({"note": cur_note}, explainer_react_template)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
async def check(note1: str, note2: str) -> bool:
    llm = ChatOpenAI(model='gpt-4o')
    sys_prompt =    r"""
                        你是一个题解检查者，你负责检查两个题解的最终答案是否一致。注意：题解不需要一模一样，
                        只要保证最后每一问的答案都完全一致即可。
                        如果答案一致：请输出是。如果答案不一致：请输出否。请确保输出一个字，不要输出“是“与”否”意外的内容

                        题解1:
                        {note1}
                        
                        题解2:
                        {note2}
                    """
    prompt = ChatPromptTemplate.from_template(sys_prompt)
    parser = StrOutputParser()
    chain = prompt | llm | parser

    response = chain.invoke(input={"note1": note1, "note2": note2})
    return response

In [ ]:
with open("output_with_answer.json", 'r') as file:
    data = json.load(file)

score = 0
is_same = [0 for _ in range(len(data))]
for i in range(len(data)):
    print(f"----- processing {i} -----")
    response = await check(data[i]['Note'], answers[i])
    if response == "是":
        score += 1
        is_same[i] = 1

score /= len(data)
score

In [ ]:
for i in range(len(is_same)):
    if is_same[i] == 0: print(i)
    

In [ ]:
import json
with open("output_with_answer.json", 'r') as file:
    data = json.load(file)

In [ ]:
problems[35]

In [ ]:
answers[35]

In [ ]:
data[35]